In [1]:
from pyspark.sql import (
    functions as f,
    SparkSession,
    types as t
)

spark = SparkSession.builder.appName("df_join").getOrCreate()

In [2]:
# user data
user_data = [
    ["1000", "Neville Hardy", "Apple"],
    ["2000", "Dacia Cohen", "Alphabet"],
    ["3000", "Elois Cox", "Neflix"],
    ["4000", "Junita Meyer", "Meta"],
    ["5000", "Cleora Banks", "Amazon"]]

user_col = ['id', 'name', 'company']
df_user = spark.createDataFrame(data=user_data, schema=user_col)
df_user.show()

# salary data
salary_data = [
    ["1000", "150000", "engineer"],
    ["2000", "240000", "manager"],
    ["3000", "120000", "human resource"],
    ["6000", "100000", "sales"]]

salary_col = ['id', 'salary', 'department']
df_salary = spark.createDataFrame(data=salary_data, schema=salary_col)
df_salary.show()

+----+-------------+--------+
|  id|         name| company|
+----+-------------+--------+
|1000|Neville Hardy|   Apple|
|2000|  Dacia Cohen|Alphabet|
|3000|    Elois Cox|  Neflix|
|4000| Junita Meyer|    Meta|
|5000| Cleora Banks|  Amazon|
+----+-------------+--------+

+----+------+--------------+
|  id|salary|    department|
+----+------+--------------+
|1000|150000|      engineer|
|2000|240000|       manager|
|3000|120000|human resource|
|6000|100000|         sales|
+----+------+--------------+



In [10]:
# inner join: join the two dataframes on common key columns.
print("== inner join ==")
df_user.join(df_salary,
               df_user.id == df_salary.id,
               "inner").show()

# inner join, then filter
df_user.join(df_salary,
               df_user.id == df_salary.id,
               "inner").filter(df_user.id == 1000).show()

# inner join, then where
df_user.join(df_salary,
               df_user.id == df_salary.id,
               "inner").where(df_user.id == 1000).show()

# multiple join with &
df_user.join(df_salary,
               (df_user.id == df_salary.id) & (df_user.id == 1000)
            ).show()

== inner join ==
+----+-------------+--------+----+------+--------------+
|  id|         name| company|  id|salary|    department|
+----+-------------+--------+----+------+--------------+
|1000|Neville Hardy|   Apple|1000|150000|      engineer|
|2000|  Dacia Cohen|Alphabet|2000|240000|       manager|
|3000|    Elois Cox|  Neflix|3000|120000|human resource|
+----+-------------+--------+----+------+--------------+

+----+-------------+-------+----+------+----------+
|  id|         name|company|  id|salary|department|
+----+-------------+-------+----+------+----------+
|1000|Neville Hardy|  Apple|1000|150000|  engineer|
+----+-------------+-------+----+------+----------+

+----+-------------+-------+----+------+----------+
|  id|         name|company|  id|salary|department|
+----+-------------+-------+----+------+----------+
|1000|Neville Hardy|  Apple|1000|150000|  engineer|
+----+-------------+-------+----+------+----------+

+----+-------------+-------+----+------+----------+
|  id|   

In [11]:
# full outer join: join the two dataframes with all matching and non-matching rows
print("== full outer join ==")
df_user.join(df_salary, 
               df_user.id == df_salary.id, 
               "fullouter").show()

== full outer join ==
+----+-------------+--------+----+------+--------------+
|  id|         name| company|  id|salary|    department|
+----+-------------+--------+----+------+--------------+
|1000|Neville Hardy|   Apple|1000|150000|      engineer|
|2000|  Dacia Cohen|Alphabet|2000|240000|       manager|
|3000|    Elois Cox|  Neflix|3000|120000|human resource|
|4000| Junita Meyer|    Meta|NULL|  NULL|          NULL|
|5000| Cleora Banks|  Amazon|NULL|  NULL|          NULL|
|NULL|         NULL|    NULL|6000|100000|         sales|
+----+-------------+--------+----+------+--------------+



In [12]:
# left join:  joins by returning all rows from the first dataframe and only matched rows from the second one
print("== left join ==")
df_user.join(df_salary, 
               df_user.id == df_salary.id, 
               "left").show()

== left join ==
+----+-------------+--------+----+------+--------------+
|  id|         name| company|  id|salary|    department|
+----+-------------+--------+----+------+--------------+
|1000|Neville Hardy|   Apple|1000|150000|      engineer|
|2000|  Dacia Cohen|Alphabet|2000|240000|       manager|
|3000|    Elois Cox|  Neflix|3000|120000|human resource|
|4000| Junita Meyer|    Meta|NULL|  NULL|          NULL|
|5000| Cleora Banks|  Amazon|NULL|  NULL|          NULL|
+----+-------------+--------+----+------+--------------+



In [16]:
# left semi join: join all rows from the first dataframe and return only matched rows from the second one
print("== left semi join ==")
df_user.join(df_salary, 
               df_user.id == df_salary.id, 
               "leftsemi").show()

# left anti join: join returns only columns from the first dataframe for non-matched records of the second dataframe
print("== left anti join ==")
df_user.join(df_salary, 
               df_user.id == df_salary.id, 
               "leftanti").show()

== left semi join ==
+----+-------------+--------+
|  id|         name| company|
+----+-------------+--------+
|1000|Neville Hardy|   Apple|
|2000|  Dacia Cohen|Alphabet|
|3000|    Elois Cox|  Neflix|
+----+-------------+--------+

== left anti join ==
+----+------------+-------+
|  id|        name|company|
+----+------------+-------+
|4000|Junita Meyer|   Meta|
|5000|Cleora Banks| Amazon|
+----+------------+-------+



In [18]:
# SQL join
df_user.createOrReplaceTempView("user")
df_salary.createOrReplaceTempView("salary")

spark.sql("SELECT * FROM user, salary WHERE user.id == salary.id").show()

spark.sql("SELECT * FROM user INNER JOIN salary ON user.id == salary.id").show()

spark.sql("SELECT * FROM user LEFT JOIN salary ON user.id == salary.id").show()

+----+-------------+--------+----+------+--------------+
|  id|         name| company|  id|salary|    department|
+----+-------------+--------+----+------+--------------+
|1000|Neville Hardy|   Apple|1000|150000|      engineer|
|2000|  Dacia Cohen|Alphabet|2000|240000|       manager|
|3000|    Elois Cox|  Neflix|3000|120000|human resource|
+----+-------------+--------+----+------+--------------+

+----+-------------+--------+----+------+--------------+
|  id|         name| company|  id|salary|    department|
+----+-------------+--------+----+------+--------------+
|1000|Neville Hardy|   Apple|1000|150000|      engineer|
|2000|  Dacia Cohen|Alphabet|2000|240000|       manager|
|3000|    Elois Cox|  Neflix|3000|120000|human resource|
+----+-------------+--------+----+------+--------------+

+----+-------------+--------+----+------+--------------+
|  id|         name| company|  id|salary|    department|
+----+-------------+--------+----+------+--------------+
|1000|Neville Hardy|   Apple|